In [10]:
# 크롤링에 필요한 패키지 모듈 불러오기 
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

from fake_useragent import UserAgent
import random

# pass word 입력값을 감추기 위한 패키지
import getpass

import os 
import requests
import shutil
import urllib
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve

# 작업 진행상황 표시를 위한 패키지 모듈 불러오기 
from tqdm import tqdm_notebook as tq
import time
import datetime

# 데이터 판다스로 정리하기 위해 판다스 패키지 불러오기 
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 랜덤으로 생성한 UserAgent 값 출력 
ua = UserAgent()
userAgent = ua.random
print(userAgent)

Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:21.0) Gecko/20130331 Firefox/21.0


In [3]:
# chrome dirver 옵션
options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={userAgent}')  # fake user 표시
# options.add_argument('headless')                 # 크롬브라우저 숨기기
options.add_argument('window-size=1920,1080')    # 모니터 해상도 지정
options.add_argument('disable-gpu')              # CPU 부담을 줄이고 GPU로 그래픽 가속

In [6]:
# 크롬드라이버 path 지정 및 옵션 지정 
driver = webdriver.Chrome(executable_path='/Users/kisung.Kim/Documents/chromedriver', chrome_options=options)

# 인스타 url 받아오기
url = 'https://www.instagram.com/'
driver.get(url)
time.sleep(3)

# 인스타 로그인 하기 - ID -> PW -> 로그인 클릭
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(input("ID를 입력하세요: "))
time.sleep(2)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(getpass.getpass ("Password를 입력하세요: "))
time.sleep(2)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(3)

ID를 입력하세요: kcs4912
Password를 입력하세요: ········


In [7]:
time_sleep = random.randint(2, 6)
keyword = input("키워드를 입력하세요: ")
url = 'https://www.instagram.com/explore/tags/'+quote_plus(keyword)
driver.get(url)
time.sleep(3)

print('스크롤 내리는 중...')
# 스크롤 내리기 
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    time_sleep = random.randint(1, 8)
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(time_sleep)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    time_sleep = random.randint(2, 9)
    if new_page_height == last_page_height:
        time.sleep(time_sleep)
        if new_page_height == driver.execute_script("return document.documentElement.scrollHeight"):
            break
    else:
        last_page_height = new_page_height
print('스크롤 완료')

print('url 링크 받는 중...')

# 인스타 정보 받기
html = driver.page_source
soup = BeautifulSoup(html)

# url 링크 추출
imglist = []
insta = soup.select('.v1Nh3.kIKUG._bz0w')
for i in insta:
    print('https://www.instagram.com' + i.a['href'])
    imgUrl = i.select_one('.KL4Bh').img['src']
    imglist.append(imgUrl)
    lmglist = list(set(imglist))
    html = driver.page_source
    soup = BeautifulSoup(html)
    insta = soup.select('.v1Nh3.kIKUG._bz0w')

키워드를 입력하세요: 여드름
스크롤 내리는 중...
스크롤 완료
url 링크 받는 중...
https://www.instagram.com/p/CaBHNEgh7Cj/
https://www.instagram.com/p/CZ2zbKdBSuQ/
https://www.instagram.com/p/CZ-K489hKVY/
https://www.instagram.com/p/CZ_gnfVF0j5/
https://www.instagram.com/p/CZ15DZGBdfU/
https://www.instagram.com/p/CaAE0jPlaPL/
https://www.instagram.com/p/CZ1BAwVle1E/
https://www.instagram.com/p/CaBlih8rl5M/
https://www.instagram.com/p/CZ2Ij2lhixg/
https://www.instagram.com/p/CZ_awH6PLob/
https://www.instagram.com/p/CZ_argmFK_z/
https://www.instagram.com/p/CZ_aVqTP1t9/
https://www.instagram.com/p/CZ_aBy4gUiN/
https://www.instagram.com/p/CZ_Z-TClpXH/
https://www.instagram.com/p/CZ_ZBOpPdQp/
https://www.instagram.com/p/CZ_Ycv1l9MN/
https://www.instagram.com/p/CZglQ--F8gd/
https://www.instagram.com/p/CZ_YC4CF0sJ/
https://www.instagram.com/p/CZ_YBIilLSG/
https://www.instagram.com/p/CZ_YCGbFBDV/
https://www.instagram.com/p/CZ_XerXlSSc/
https://www.instagram.com/p/CZ_Xc7qljrL/
https://www.instagram.com/p/CZ_Xd2-FPZx/
https:

In [8]:
if not os.path.isdir('/Users/kisung.Kim/Documents/kimkisung/Work/Dermal_Freeco_Project/img_data_collection/insta_img'):
    print('폴더 생성 완료')
    os.mkdir('/Users/kisung.Kim/Documents/kimkisung/Work/Dermal_Freeco_Project/img_data_collection/insta_img')
else:
    print('이미 동일한 이름의 폴더가 있습니다.')

폴더 생성 완료


In [12]:
print("이미지 추출을 시작합니다.")
n=0

for i in tq(range(len(lmglist))):
    # This is the image url.
    image_url = imglist[n]
    # Open the url image, set stream to True, this will return the stream content.
    resp = requests.get(image_url, stream=True)
    # Open a local file with wb ( write binary ) permission.
    local_file = open('./insta_img/' + keyword + str(n) + '.jpg', 'wb')
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    resp.raw.decode_content = True
    # Copy the response stream raw data to local image file.
    shutil.copyfileobj(resp.raw, local_file)
    # Remove the image url response object.
    n +=1
    del resp

print("이미지 추출이 완료되었습니다.")

이미지 추출을 시작합니다.


  0%|          | 0/42 [00:00<?, ?it/s]

이미지 추출이 완료되었습니다.


In [13]:
result_df = pd.DataFrame()
result_df['link'] = imglist

now = datetime.datetime.now()
now = str(now).replace(' ','')

result_df.to_csv('./insta_img/' + f'키워드_{keyword}instar{now}.csv', index=False)
driver.close()